<a href="https://colab.research.google.com/github/ak9250/fewshot-face-translation-GAN/blob/master/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab commands

The following cells will download prerequisite for fewshot-face-translation-GAN.

In [0]:
!git clone --recursive https://github.com/shaoanlu/fewshot-face-translation-GAN.git

In [0]:
# There are import errors under keras == 2.2.5
!pip install keras==2.2.4

In [0]:
%cd "fewshot-face-translation-GAN"

In [0]:
# Download pre-trined weights
!gdown https://drive.google.com/uc?id=1DUMmZGTGKMyEYSKy-w34IDHawVF24rIs
!gdown https://drive.google.com/uc?id=1xl8cg7xaRnMsyiODcXguJ83d5hwodckB

In [0]:
!mkdir weights
!mv decoder.h5 weights/decoder.h5
!mv encoder.h5 weights/encoder.h5

In [0]:
!ls

## Load GAN model

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
from models import FaceTranslationGANInferenceModel

In [0]:
model = FaceTranslationGANInferenceModel()

## Load face-toolbox

In [0]:
from face_toolbox_keras.models.verifier.face_verifier import FaceVerifier
fv = FaceVerifier(classes=512)

from face_toolbox_keras.models.parser import face_parser
fp = face_parser.FaceParser()

from face_toolbox_keras.models.detector import face_detector
fd = face_detector.FaceAlignmentDetector()

from face_toolbox_keras.models.detector.iris_detector import IrisDetector
idet = IrisDetector()
#idet.set_detector(fd)

# Translate faces

In [0]:
import numpy as np
from utils import utils
from matplotlib import pyplot as plt

use youtube-dl for video face swapping

In [0]:
!pip install youtube-dl

In [0]:
!youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' url to youtube video here

In [0]:
mkdir raw_images

In [0]:
!ffmpeg -i youryoutubevideotitle.mp4 raw_images/$filename%03d.png

In [0]:
!wget -O /content/fewshot-face-translation-GAN/test.jpg url to test image

### Inference

It requires additional time to load models for the first inference.


In [0]:
import os
import os.path
import shutil
from PIL import Image

target_path = '/content/fewshot-face-translation-GAN/test.jpg'
tar, emb_tar = utils.get_tar_inputs([target_path], fd, fv)
src_dir = "/content/fewshot-face-translation-GAN/raw_images/"
result_dir = "/content/fewshot-face-translation-GAN/results/"
os.makedirs(result_dir, exist_ok=True)

for target_name in os.listdir(src_dir):
  src_path = os.path.join(src_dir, target_name)
  result_path = os.path.join(result_dir, target_name)
  print(src_path)
  try:
    src, mask, aligned_im, (x0, y0, x1, y1), landmarks = utils.get_src_inputs(src_path, fd, fp, idet)
  except IndexError as e:
    print("get_src_inputs IndexError:", e)
    shutil.copyfile(src_path, result_path)
    continue
  except AssertionError as e:
    print("get_src_inputs AssertionError:", e)
    shutil.copyfile(src_path, result_path)
    continue
    
  out = model.inference(src, mask, tar, emb_tar)
  result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
  result_img = utils.post_process_result(src_path, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks)
  Image.fromarray(result_img).save(result_path)

### Visualize results

see image result or output a video mp4 of the result images

In [0]:
plt.imshow(result_img)

In [0]:
!ffmpeg -f image2 -i /content/fewshot-face-translation-GAN/results/$filename%03d.png -vf scale=256:256 -vcodec libx264 -pix_fmt yuv420p -strict -2 -acodec aac video.mp4 -y